In [ ]:
import gradio as gr
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Sample dataset (replace with real dataset)
data = {
    "Size (sq ft)": [1500, 2000, 2500, 1800, 2200, 1700, 1600, 2400, 2100, 2300],
    "Bedrooms": [3, 4, 4, 3, 5, 3, 2, 4, 4, 5],
    "Age (years)": [10, 15, 20, 5, 8, 12, 6, 14, 18, 9],
    "Price ($)": [300000, 400000, 500000, 350000, 450000, 320000, 280000, 480000, 420000, 460000]
}

df = pd.DataFrame(data)

# Categorizing Prices (0: Affordable, 1: Expensive, 2: Luxury)
def categorize_price(price):
    if price < 350000:
        return 0
    elif price <= 450000:
        return 1
    else:
        return 2

df["Price Category"] = df["Price ($)"].apply(categorize_price)
X = df.drop(columns=["Price ($)", "Price Category"])
y = df["Price Category"]

# Scaling the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train a Logistic Regression model
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)

def predict_category(size, bedrooms, age):
    input_data = np.array([[size, bedrooms, age]])
    input_scaled = scaler.transform(input_data)
    category = model.predict(input_scaled)[0]
    categories = {0: "Affordable", 1: "Expensive", 2: "Luxury"}
    return f"Estimated Category: {categories[category]}"

# Gradio Interface
demo = gr.Interface(
    fn=predict_category,
    inputs=[
        gr.Number(label="Size (sq ft)"),
        gr.Number(label="Bedrooms"),
        gr.Number(label="Age (years)")
    ],
    outputs=gr.Textbox(label="Predicted Category"),
    title="House Price Category Prediction AI",
    description="Enter house details to get a predicted price category."
)

demo.launch()
